# Using Obspy plot the noise floor

To use obspy generate teh noise floor test is relatively easy, only 3 steps:

(1) Get the waveform from NCEDC

(2) Get the pole and zero information for this station

(3) Get the noise floor

The method used here is:

McNamara, D. E. and Buland, R. P. (2004),
Ambient Noise Levels in the Continental United States,
Bulletin of the Seismological Society of America, 94 (4), 1517-1527

### Let's first import useful packages, and also the parse_ncedc.py and build_url.py in the current folder

In [1]:
#%pylab inline
from obspy.fdsn import Client
from obspy import UTCDateTime
import parse_ncedc as pn
import build_url as bu
from obspy.signal import PPSD

/Library/Python/2.7/site-packages/numpy-override/numpy/oldnumeric/__init__.py:11: ModuleDeprecationWarning: The oldnumeric module will be dropped in Numpy 1.9
  warnings.warn(_msg, ModuleDeprecationWarning)


In [5]:
Client??

### (1) Get the waveform

In [2]:
#create an Client object
client = Client('NCEDC')

#specify the start and end time and other station info
t1 = UTCDateTime("2014-02-27T18:52:00.000")
t2 = UTCDateTime("2014-02-27T18:57:00.000")
network = 'PG'
station = 'OBS3'
location = '*'
channel = 'HH1'

#get the waveform as a stream object
st = client.get_waveforms(network, station, location, channel, t1,t2)

#plot the waveform
st.plot()

#get the first trace of the waveform from the stream
tr = st[0]

/Library/Python/2.7/site-packages/obspy/fdsn/wadl_parser.py:106: UserWarning: The 'dataselect' service at 'http://service.ncedc.org/fdsnws/dataselect/1/' cannot deal with the following required parameters: quality, minimumlength, longestonly
They will not be available for any requests. Any attempt to use them will result in an error.
  warnings.warn(msg)
/Library/Python/2.7/site-packages/obspy/fdsn/wadl_parser.py:106: UserWarning: The 'station' service at 'http://service.ncedc.org/fdsnws/station/1/' cannot deal with the following required parameters: includerestricted
They will not be available for any requests. Any attempt to use them will result in an error.
  warnings.warn(msg)
/Library/Python/2.7/site-packages/obspy/fdsn/wadl_parser.py:106: UserWarning: The 'event' service at 'http://service.ncedc.org/fdsnws/event/1/' cannot deal with the following required parameters: includeallorigins, updatedafter
They will not be available for any requests. Any attempt to use them will result i

In [4]:
t1 = UTCDateTime("2014-02-12T09:25:00.000")
t2 = UTCDateTime("2014-02-12T12:30:00.000")
tr.plot(starttime = t1, endtime = t2)

IndexError: list index out of range

### (2) Get the pole and zero information of the station

In [3]:
#get the dataless seed file from NCEDC, I also printed out the url you need to find this file
seed = bu.get_dataless_seed(network, station, location, channel, t1,t2)

#parse the dataless seed to get the pole and zero info
parser = pn.Parser_ncedc(seed)
paz = parser.getPAZ(tr.id)


http://service.ncedc.org/ncedcws/dataless/1/query?loc=%2A&end=2014-02-27T18%3A57%3A00&sta=OBS3&cha=HH1&start=2014-02-27T18%3A52%3A00&net=PG


In [5]:
print paz

{'sensitivity': 468750000.0, 'digitizer_gain': 312500.0, 'seismometer_gain': 1500.0, 'zeros': [0j, 0j], 'gain': 571572000.0, 'poles': [(-0.00588873+0.00588873j), (-0.00588873-0.00588873j), (-502.655+0j), (-1005.31+0j), (-1130.97+0j)]}


### (3) Calculate the noise floor

In [23]:
#Initialize a new PPSD instance. The ppsd object will then make sure that only appropriate 
#data go into the probabilistic psd statistics. Then get the noise floor
ppsd = PPSD(tr.stats, paz, db_bins=(-200, 10, 1.))
ppsd.add(st)
ppsd.plot()

In [24]:
ppsd.hist_stack[56]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [10]:
#the following commend will give you all the data used to plot the noise floor
ppsd.hist_stack

#it has 112 array corresponding to 112 center frequency, in each of the array, it has 150 elements corresponding to the amplitude bin
len(ppsd.hist_stack[0])


210

### A couple of useful things

In [ ]:
ppsd

In [ ]:
from obspy.xseed import Parser

In [ ]:
ppsd??

In [17]:
%load ./parse_ncedc.py

In [ ]:
%%bash?

In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %colors  %config  %connect_info  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %install_default_config  %install_ext  %install_profiles  %killbgscripts  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %lsmagic  %macro  %magic  %man  %matplotlib  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %run  %save  %sc  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%latex  %%perl  %%prun  %%pypy  %%python  %%python3  %%ruby  %%script  %%sh  %%svg  %%sx 

In [18]:
!ls

Obspy_noise_floor.ipynb build_url.py            parse_ncedc.py
Readme.txt              build_url.pyc           parse_ncedc.pyc


In [19]:
!pwd

/Users/kongqk/Research/Berkeley/python_mini_workshop/01_noise_floor_obspy_20140210


## Very nice gallery of useful ipython notebooks

https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks

In [ ]:
%pwd